In [1]:
import os
import shutil
import glob
import json
import time
import sys
from rm_tools.rm2svg import rm2svg
from PyPDF2 import PdfFileReader

## Set this up to work with the test example

There is one .rm file for every page. Each has its own hash. To find out the order we look into the content file. 

In [2]:
refNrPath = "paper"
origPDF = "paper.pdf"
meta = json.loads(open(refNrPath + ".metadata").read())
content = json.loads(open(refNrPath + ".content").read())
fname = meta["visibleName"]
input1 = PdfFileReader(open(origPDF, "rb"))
npages = input1.getNumPages()
pdfsize = input1.getPage(0).mediaBox
pdfx = int(pdfsize[2])
pdfy = int(pdfsize[3])

In old versions empty pages don't generate \*.rm files. So we just use a placeholder empty file to simulate it.

In [3]:
#os.mkdir("tempDir")
emptyRm = "empty.rm"
rm2svg(emptyRm, "tempDir/emptyrm.svg", coloured_annotations=False, x_width=pdfx, y_width=pdfy)

## 1.+2. Loop through the number of pages, creating an annotation pdf for each.

In [4]:
pdflist = []
for pg, pg_hash in enumerate(content['pages']):
    # print(pg)
    rmpath = refNrPath + "/" + pg_hash + ".rm"
    if os.path.isfile(rmpath):
        rm2svg(rmpath, "tempDir/temprm" + str(pg) + ".svg", coloured_annotations=False, x_width=pdfx, y_width=pdfy)
        svg_path = "tempDir/temprm" + str(pg) + ".svg"
    else:
        svg_path = "tempDir/emptyrm.svg"
    convertSvg2PdfCmd = "".join(["rsvg-convert -f pdf -o ", "tempDir/temppdf" + str(pg), ".pdf ", svg_path])
    os.system(convertSvg2PdfCmd)
    pdflist.append("tempDir/temppdf"+str(pg)+".pdf")

## 3. merge all pdf pages into one

In [5]:
merged_rm = "tempDir/merged_rm.pdf"
os.system("convert "+ (" ").join(pdflist)+" "+merged_rm)

0

## 4. overlay on top of the existing page

In [6]:
stampCmd = "".join(["pdftk ", origPDF, " multistamp ", merged_rm, " output ", origPDF[:-4], "_annot.pdf"])
os.system(stampCmd)

0